**This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/missing-values).**

---


Now it's your turn to test your new knowledge of **missing values** handling. You'll probably find it makes a big difference.

# Setup

The questions will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex2 import *
print("Setup Complete")

Setup Complete


In this exercise, you will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://storage.googleapis.com/kaggle-media/learn/images/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
X.columns, len(X.columns.tolist())

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
        'MoSold', 'YrSold'],
       dtype='object'),
 36)

In [4]:
y.name

'SalePrice'

In [5]:
X_test.columns, len(X_test.columns.tolist())

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
        'MoSold', 'YrSold'],
       dtype='object'),
 36)

Use the next code cell to print the first five rows of the data.

You can already see a few missing values in the first several rows.  In the next step, you'll obtain a more comprehensive understanding of the missing values in the dataset.

# Step 1: Preliminary investigation

Run the code cell below without changes.

In [6]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


### Part A

Use the above output to answer the questions below.

In [7]:
# Fill in the line below: How many rows are in the training data?
num_rows = X_train.shape[0]

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = len(missing_val_count_by_column[missing_val_count_by_column > 0])

# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = sum(missing_val_count_by_column)

# Check your answers
step_1.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
step_1.a.hint()
step_1.a.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use the output of `X_train.shape` to get the number of rows and columns in the training data.  The `missing_val_count_by_column` Series has an entry for each column in the data, and the output above prints the number of missing entries for each column with at least one missing entry.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# How many rows are in the training data?
num_rows = 1168

# How many columns in the training data have missing values?
num_cols_with_missing = 3

# How many missing entries are contained in all of the training data?
tot_missing = 212 + 6 + 58

```

### Part B
Considering your answers above, what do you think is likely the best approach to dealing with the missing values?

In [9]:
# Check your answer (Run this code cell to receive credit!)
"Imputation"
step_1.b.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results.  This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

In [10]:
step_1.b.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Does the dataset have a lot of missing values, or just a few?  Would we lose much information if we completely ignored the columns with missing entries?

To compare different approaches to dealing with missing values, you'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 2: Drop columns with missing values

In this step, you'll preprocess the data in `X_train` and `X_valid` to remove columns with missing values.  Set the preprocessed DataFrames to `reduced_X_train` and `reduced_X_valid`, respectively.  

In [12]:
# Fill in the line below: get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()] # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# Check your answers
step_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# Lines below will give you a hint or solution code
step_2.hint()
step_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Begin by finding the list of columns in the data with missing values.  Then, drop these columns in both the training and validation data with the `drop()` method.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

```

Run the next code cell without changes to obtain the MAE for this approach.

In [14]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


# Step 3: Imputation

### Part A

Use the next code cell to impute missing values with the mean value along each column.  Set the preprocessed DataFrames to `imputed_X_train` and `imputed_X_valid`.  Make sure that the column names match those in `X_train` and `X_valid`.

In [15]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

# Check your answers
step_3.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [16]:
# Lines below will give you a hint or solution code
#step_3.a.hint()
#step_3.a.solution()

Run the next code cell without changes to obtain the MAE for this approach.

In [17]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


### Part B

Compare the MAE from each approach.  Does anything surprise you about the results?  Why do you think one approach performed better than the other?

In [18]:
# Check your answer (Run this code cell to receive credit!)
"imputation of means might not be the most accurate"
step_3.b.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

Given that thre are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely.  However, we see that dropping columns performs slightly better!  While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset.  That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method.  For instance, consider the `GarageYrBlt` column (which indicates the year that the garage was built).  It's likely that in some cases, a missing value could indicate a house that does not have a garage.  Does it make more sense to fill in the median value along each column in this case?  Or could we get better results by filling in the minimum value along each column?  It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!

In [19]:
step_3.b.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Did removing missing values yield a larger or smaller MAE than imputation? Does this agree with the coding example from the tutorial?

# Step 4: Generate test predictions

In this final step, you'll use any approach of your choosing to deal with missing values.  Once you've preprocessed the training and validation features, you'll train and evaluate a random forest model.  Then, you'll preprocess the test data before generating predictions that can be submitted to the competition!

### Part A

Use the next code cell to preprocess the training and validation data.  Set the preprocessed DataFrames to `final_X_train` and `final_X_valid`.  **You can use any approach of your choosing here!**  in order for this step to be marked as correct, you need only ensure:
- the preprocessed DataFrames have the same number of columns,
- the preprocessed DataFrames have no missing values, 
- `final_X_train` and `y_train` have the same number of rows, and
- `final_X_valid` and `y_valid` have the same number of rows.

In [20]:
X_train.shape, X_valid.shape

((1168, 36), (292, 36))

In [21]:
y_train.shape, y_valid.shape

((1168,), (292,))

In [22]:
# Strategy: Find the features with low correlation to SalePrice and drop them
correlation_matrix = X_train.corrwith(y_train)
low_correlation_cols = correlation_matrix[abs(correlation_matrix) < 0.04].index
print(low_correlation_cols)
# Copying the original DataFrames to new ones without modifying the originals
final_X = X.drop(columns=low_correlation_cols)
final_X_train = X_train.drop(columns=low_correlation_cols)
final_X_valid = X_valid.drop(columns=low_correlation_cols)
final_X_test = X_test.drop(columns=low_correlation_cols)

Index(['BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', 'PoolArea', 'MiscVal'], dtype='object')


In [23]:
X.shape, X_train.shape, X_valid.shape, X_test.shape

((1460, 36), (1168, 36), (292, 36), (1459, 36))

In [24]:
final_X.shape, final_X_train.shape, final_X_valid.shape, final_X_test.shape

((1460, 31), (1168, 31), (292, 31), (1459, 31))

In [25]:
final_X_train.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MoSold',
       'YrSold'],
      dtype='object')

In [26]:
# Strategy: Identify highly correlated columns in X and drop one column of each pair
correlation_matrix = X_train.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

highly_correlated_pairs = []
for i in range(len(upper_triangle.columns)):
    for j in range(i+1, len(upper_triangle.columns)):
        if upper_triangle.iloc[i, j] > 0.80:
            highly_correlated_pairs.append((upper_triangle.columns[i], upper_triangle.columns[j]))

print(highly_correlated_pairs)

# Extracting the second element of each pair
features_to_drop = [pair[1] for pair in highly_correlated_pairs]
print(features_to_drop)

# Dropping the features from X, X_train, X_valid, and X_test
final_X.drop(columns=features_to_drop, inplace=True)
final_X_train.drop(columns=features_to_drop, inplace=True)
final_X_valid.drop(columns=features_to_drop, inplace=True)
final_X_test.drop(columns=features_to_drop, inplace=True)

[('YearBuilt', 'GarageYrBlt'), ('GrLivArea', 'TotRmsAbvGrd'), ('GarageCars', 'GarageArea')]
['GarageYrBlt', 'TotRmsAbvGrd', 'GarageArea']


In [27]:
X.shape, X_train.shape, X_valid.shape, X_test.shape

((1460, 36), (1168, 36), (292, 36), (1459, 36))

In [28]:
final_X.shape, final_X_train.shape, final_X_valid.shape, final_X_test.shape

((1460, 28), (1168, 28), (292, 28), (1459, 28))

In [29]:
final_X_train.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces',
       'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'MoSold', 'YrSold'],
      dtype='object')

In [30]:
# Strategy to fill missing values in GarageYrBlt:
# Fill in with the sum of YearBuilt and the average of 
# the difference between GarageYrBlt and YearBuilt

# Calculate the mean difference between GarageYrBlt and YearBuilt
mean_diff = (X_train['GarageYrBlt'] - X_train['YearBuilt']).mean()
print(f"The mean difference between GarageYrBlt and YearBuilt is {mean_diff:.1f} years.")

# Function to fill missing values in GarageYrBlt column
def fill_garage_year(row):
    if pd.isnull(row['GarageYrBlt']):
        return row['YearBuilt'] + mean_diff
    else:
        return row['GarageYrBlt']

The mean difference between GarageYrBlt and YearBuilt is 5.7 years.


In [31]:
# Strategy to fill missing values in LotFrontage:
# Fill in with the product of LotArea and the average of 
# the division between LotFrontage and LotArea

# Calculate the mean of the division between LotFrontage and LotArea
mean_division = (X_train['LotFrontage'] / X_train['LotArea']).mean()
print(f"The mean division between LotFrontage and LotArea is {mean_division:.6f}.")

# Function to fill missing values in GarageYrBlt column
def fill_lot_frontage(row):
    if pd.isnull(row['LotFrontage']):
        return row['LotArea'] * mean_division
    else:
        return row['LotFrontage']

The mean division between LotFrontage and LotArea is 0.007866.


In [32]:
# Strategy: 
# Combining MoSold and YrSold into a single numerical column YearSold and then 
# transforming the year columns can be a good approach. 
# This way, we're effectively reducing the dimensionality of the dataset while 
# still capturing the temporal information.

def transform_year_columns(X_df):
    # Combine MoSold and YrSold into a single numerical column: YearSold
    X_df['YearSold'] = X_df['YrSold'] + X_df['MoSold'] / 12
    X_df.drop(['MoSold', 'YrSold'], axis=1, inplace=True)

    # Transform YearBuilt to PropertyAge
    X_df['PropertyAge'] = X_df['YearSold'] - X_df['YearBuilt']
    X_df.drop('YearBuilt', axis=1, inplace=True)

    # Transform YearRemodAdd to RemodAge
    X_df['RemodAge'] = X_df['YearSold'] - X_df['YearRemodAdd']
    X_df.drop('YearRemodAdd', axis=1, inplace=True)

#     # Transform GarageYrBlt to GarageAge
#     X_df['GarageAge'] = X_df['YearSold'] - X_df['GarageYrBlt']
#     X_df.drop('GarageYrBlt', axis=1, inplace=True)

    return X_df


In [33]:
# Preprocessed training and validation features
missing_val_count_by_column = (final_X_train.isnull().sum())
print("missing values before preprocessing:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

# # Fill in missing values in GarageYrBlt
# final_X_train['GarageYrBlt'] = final_X_train.apply(fill_garage_year, axis=1)
# final_X_valid['GarageYrBlt'] = final_X_valid.apply(fill_garage_year, axis=1)
# missing_val_count_by_column = (final_X_train.isnull().sum())
# print("missing values after filling GarageYrBlt:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

# Fill in missing values in LotFrontage
final_X['LotFrontage'] = final_X.apply(fill_lot_frontage, axis=1)
final_X_train['LotFrontage'] = final_X_train.apply(fill_lot_frontage, axis=1)
final_X_valid['LotFrontage'] = final_X_valid.apply(fill_lot_frontage, axis=1)
final_X_test['LotFrontage'] = final_X_test.apply(fill_lot_frontage, axis=1)
missing_val_count_by_column = (final_X_train.isnull().sum())
print("missing values after filling LotFrontage:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

# Save column names before Imputation
final_X_train_columns = final_X_train.columns
final_X_valid_columns = final_X_valid.columns
final_X_test_columns = final_X_test.columns
final_X_columns = final_X.columns

# Apply Imputation
final_imputer = SimpleImputer()
final_X_train = pd.DataFrame(final_imputer.fit_transform(final_X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(final_X_valid))
final_X_test = pd.DataFrame(final_imputer.transform(final_X_test))
final_X = pd.DataFrame(final_imputer.transform(final_X))

# Fill in the lines below: imputation removed column names; put them back
final_X_train.columns = final_X_train_columns
final_X_valid.columns = final_X_valid_columns
final_X_test.columns = final_X_test_columns
final_X.columns = final_X_columns

missing_val_count_by_column = (final_X_train.isnull().sum())
print("missing values after Imputation:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

# Transforming the year columns
final_X_train = transform_year_columns(final_X_train)
final_X_valid = transform_year_columns(final_X_valid)
final_X_test = transform_year_columns(final_X_test)
final_X = transform_year_columns(final_X)

# Check your answers
step_4.a.check()

missing values before preprocessing:
LotFrontage    212
MasVnrArea       6
dtype: int64

missing values after filling LotFrontage:
MasVnrArea    6
dtype: int64

missing values after Imputation:
Series([], dtype: int64)



<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [34]:
final_X_train.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'YearSold', 'PropertyAge', 'RemodAge'],
      dtype='object')

In [35]:
final_X_train.shape, final_X_valid.shape

((1168, 27), (292, 27))

In [36]:
# check if final_X_train and y_train have the same number of rows.
print(final_X_train.shape[0] == y_train.shape[0])

# check if final_X_valid and y_valid have the same number of rows.
print(final_X_valid.shape[0] == y_valid.shape[0])

True
True


In [37]:
# Lines below will give you a hint or solution code
step_4.a.hint()
step_4.a.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use any approach of your choosing to deal with missing values in the data.  For inspiration, check out the code from the tutorial!

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

```

Run the next code cell to train and evaluate a random forest model.  (*Note that we don't use the `score_dataset()` function above, because we will soon use the trained model to generate test predictions!*)

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 150, 300],  # you can adjust the range
    'max_depth': [None, 10, 20, 30, 40, 50]  # you can adjust the range
}

# Instantiate the RandomForestRegressor
rf = RandomForestRegressor(random_state=0)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(final_X_train, y_train)

# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

# Get the best estimator
best_rf = grid_search.best_estimator_

# Train the model with the best parameters
best_rf.fit(final_X_train, y_train)


Best Parameters: {'max_depth': None, 'n_estimators': 100}


RandomForestRegressor(random_state=0)

In [39]:
# Get validation predictions and MAE
preds_valid = best_rf.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17453.36412671233


In [40]:
len(preds_valid)

292

In [41]:
# # Define and fit model
# model = RandomForestRegressor(n_estimators=35, random_state=0)
# model.fit(final_X_train, y_train)

# # Get validation predictions and MAE
# preds_valid = model.predict(final_X_valid)
# print("MAE (Your approach):")
# print(mean_absolute_error(y_valid, preds_valid))

### Part B

Use the next code cell to preprocess your test data.  Make sure that you use a method that agrees with how you preprocessed the training and validation data, and set the preprocessed test features to `final_X_test`.

Then, use the preprocessed test features and the trained model to generate test predictions in `preds_test`.

In order for this step to be marked correct, you need only ensure:
- the preprocessed test DataFrame has no missing values, and
- `final_X_test` has the same number of rows as `X_test`.

In [42]:
# Fill in the line below: preprocess test data
# It was done lines above
# Apply the custom function to fill missing values
missing_val_count_by_column = (final_X_test.isnull().sum())
print("missing values before Preprocessing:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

# Fill in the line below: get test predictions
preds_test = best_rf.predict(final_X_test)

# Check your answers
step_4.b.check()

missing values before Preprocessing:
Series([], dtype: int64)



<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [43]:
final_X_test.shape, X_test.shape

((1459, 27), (1459, 36))

In [44]:
# Check if final_X_test has the same number of rows as X_test
final_X_test.shape[0] == X_test.shape[0]

True

In [45]:
# Lines below will give you a hint or solution code
step_4.b.hint()
step_4.b.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> After preprocessing the test data, you can get the model's predictions by using `model.predict()`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# Preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Get test predictions
preds_test = model.predict(final_X_test)

```

Run the next code cell without changes to save your results to a CSV file that can be submitted directly to the competition.

In [46]:
# # Save test predictions to file
# output = pd.DataFrame({'Id': X_test.index,
#                        'SalePrice': preds_test})
# output.to_csv('submission.csv', index=False)
# output

In [47]:
# Preprocessed training and validation features
# We will use all the data in final_X, which was preprocessed lines above
missing_val_count_by_column = (final_X.isnull().sum())
print("missing values before preprocessing:", "\n", missing_val_count_by_column[missing_val_count_by_column > 0], sep="", end="\n\n")

missing values before preprocessing:
Series([], dtype: int64)



In [48]:
final_X.shape, y.shape

((1460, 27), (1460,))

In [49]:
# Train the model with the best parameters, usin the complete final_X and y
best_rf.fit(final_X, y)

RandomForestRegressor(random_state=0)

In [50]:
# # Fill in the line below: get test predictions
preds_test = best_rf.predict(final_X_test)
len(preds_test)

1459

In [51]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission_full_data53.csv', index=False)
output

,Id,SalePrice
0,1461,127473.66
1,1462,150754.75
2,1463,179008.54
3,1464,181066.00
4,1465,200508.19
...,...,...
1454,2915,81991.11
1455,2916,83992.11
1456,2917,164261.59
1457,2918,115416.00


In [52]:
# check if final_X and y have the same number of rows.
print(final_X.shape[0] == y.shape[0])

True


# Submit your results

Once you have successfully completed Step 4, you're ready to submit your results to the leaderboard!  (_You also learned how to do this in the previous exercise.  If you need a reminder of how to do this, please use the instructions below._)  

First, you'll need to join the competition if you haven't already.  So open a new window by clicking on [this link](https://www.kaggle.com/c/home-data-for-ml-course).  Then click on the **Join Competition** button.

![join competition image](https://storage.googleapis.com/kaggle-media/learn/images/wLmFtH3.png)

Next, follow the instructions below:
1. Begin by clicking on the **Save Version** button in the top right corner of the window.  This will generate a pop-up window.  
2. Ensure that the **Save and Run All** option is selected, and then click on the **Save** button.
3. This generates a window in the bottom left corner of the notebook.  After it has finished running, click on the number to the right of the **Save Version** button.  This pulls up a list of versions on the right of the screen.  Click on the ellipsis **(...)** to the right of the most recent version, and select **Open in Viewer**.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the **Data** tab near the top of the screen.  Then, click on the file you would like to submit, and click on the **Submit** button to submit your results to the leaderboard.

You have now successfully submitted to the competition!

If you want to keep working to improve your performance, select the **Edit** button in the top right of the screen. Then you can change your code and repeat the process. There's a lot of room to improve, and you will climb up the leaderboard as you work.


# Keep going

Move on to learn what **[categorical variables](https://www.kaggle.com/alexisbcook/categorical-variables)** are, along with how to incorporate them into your machine learning models.  Categorical variables are very common in real-world data, but you'll get an error if you try to plug them into your models without processing them first!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*